In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
def scrape_x_posts(username, nbr):
    posts = []
    count = 0
    base_url = f"https://nitter.net/{username}"
    url = base_url

    # Using chrome to load dynamically the pages
    driver = webdriver.Chrome()
    
    while count < nbr:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        divs = soup.find_all("div", class_="timeline-item")

        if not divs:
            print("Unable to load tweets.")
            break

        for div in divs:
            if count == nbr:
                break

            # Retrieving tweet link
            a_tag = div.find("a", class_="tweet-link")
            if not a_tag:
                continue
            
            link = "https://nitter.net" + a_tag.get("href")
                
            # Avoiding retweets
            if username in link:
                # Retrieving date
                span_tag = div.find("span", class_="tweet-date")
                a_date_tag = span_tag.find("a") if span_tag else None
                publication_date = a_date_tag["title"] if a_date_tag else "unknown"
                    
                # Retrieving tweet content
                content_div = div.find("div", class_="tweet-content media-body")
                content = content_div.get_text(strip=True) if content_div else ""

                # Adding tweet to list
                posts.append({
                    "link": link,
                    "publication_date": publication_date,
                    "content": content
                })
                count += 1

        # Handling page transitions
        show_more = soup.find("div", class_="show-more", string=lambda text: text and "Load more" in text)
        next_link_tag = show_more.find("a") if show_more else None
        if next_link_tag:
            url = base_url + next_link_tag["href"]
        else:
            break
    
    driver.quit()
    return posts

In [7]:
posts = scrape_x_posts("ylecun", 10)

In [9]:
posts

[{'link': 'https://nitter.net/ylecun/status/1862598063275061484#m',
  'publication_date': 'Nov 29, 2024 · 8:42 PM UTC',
  'content': "I do not write posts on X.\nI tweet links to posts on other platforms.\nI like and retweet (occasionally)\nI comment on friends' tweets (rarely)\nFollow me on...⬇️⬇️⬇️"},
 {'link': 'https://nitter.net/ylecun/status/1916482680012849162#m',
  'publication_date': 'Apr 27, 2025 · 1:20 PM UTC',
  'content': 'Starting now.eventbrite.sg/e/nus120-disti…'},
 {'link': 'https://nitter.net/ylecun/status/1913609805467369718#m',
  'publication_date': 'Apr 19, 2025 · 3:05 PM UTC',
  'content': 'Excellentes recommandations de la part de l\'ancien ministre des finances Bruno Le Maire.\n"Les Européens se trouvent devant une alternative simple : investir ou mourir. Investir dans l’IA pour améliorer notre productivité et donc offrir enfin de meilleurs salaires à ceux qui travaillent, investir dans la défense pour garantir notre sécurité,..."'},
 {'link': 'https://nitter.net